<a href="https://colab.research.google.com/github/ndeyadieng1907/chatbot-en-cours/blob/main/Copie_de_Langchain_Agent_with_Mistral7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install accelerate
!pip install bitsandbytes

NOTE: You might need to restart the session after the pip install block

In [3]:
import warnings
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.models.mistral.modeling_mistral import MistralForCausalLM
from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast

In [4]:
# prompt: get my hugging face token from google variable env key
import os
HF_TOKEN = os.getenv("HF_KEY")

In [5]:
warnings.filterwarnings('ignore')

In [6]:
# prompt: get my hugging face token from google variable env key
import os
HF_TOKEN = os.getenv("HF_KEY")

In [7]:
# prompt: connect to huggingface

from huggingface_hub import login

login(token=HF_TOKEN)


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Configuration de la quantification pour réduire l'utilisation de la mémoire (4 bits)
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# Chargement du modèle avec précision bfloat16 et configuration de quantification
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # Ajout de la virgule manquante
    quantization_config=quantization_config,
    device_map="auto"
)

# Utilisation de bfloat16 pour des performances optimisées
# Application de la configuration de quantification
# Allocation automatique sur les GPU disponibles

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Préparation du tokenizer pour le traitement des textes


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"# Configuration de la quantification pour réduire l'utilisation de la mémoire (4 bits)

quantization_config = BitsAndBytesConfig(load_in_4bit=True)# Chargement du modèle avec précision bfloat16 et configuration de quantification
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=quantization_config, device_map="auto")# Utilisation de bfloat16 pour des performances optimisées,Application de la configuration de quantification,Allocation automatique sur les GPU disponibles
tokenizer = AutoTokenizer.from_pretrained(model_name)# Préparation du tokenizer pour le traitement des textes

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import Optional, List, Mapping, Any

In [11]:
# Importation de la classe de base LLM (Large Language Model) depuis LangChain
# Importation du gestionnaire de rappels pour la gestion des exécutions de LLM
# Importation des types optionnels et des collections pour la typage statique

In [12]:
# Définition d'une classe personnalisée CustomLLMMistral qui hérite de la classe LLM
class CustomLLMMistral(LLM):
    # Déclaration d'un attribut pour le modèle Mistral utilisé pour la génération de texte
    model: MistralForCausalLM

    # Déclaration d'un attribut pour le tokenizer Llama qui sera utilisé pour encoder les entrées
    tokenizer: LlamaTokenizerFast

    # Propriété qui retourne le type de LLM, ici spécifié comme "custom"
    @property
    def _llm_type(self) -> str:
        return "custom"

    # Méthode principale qui appelle le modèle avec un prompt donné et des options supplémentaires
    def _call(self, prompt: str, stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:

        # Création d'une liste de messages avec le rôle "user" et le contenu du prompt
        messages = [
            {"role": "user", "content": prompt},
        ]

        # Application du template de chat sur les messages et encodage en tenseurs PyTorch
        encodeds = self.tokenizer.apply_chat_template(messages, return_tensors="pt")

        # Transfert des entrées encodées vers le même appareil que le modèle (CPU ou GPU)
        model_inputs = encodeds.to(self.model.device)

        # Génération d'identifiants de tokens à partir des entrées du modèle avec des paramètres spécifiques
        generated_ids = self.model.generate(
            model_inputs,
            max_new_tokens=512,  # Nombre maximum de nouveaux tokens à générer
            do_sample=True,  # Activation du mode d'échantillonnage pour la génération
            pad_token_id=self.tokenizer.eos_token_id,  # ID du token de remplissage (fin de séquence)
            top_k=4,  # Limitation aux 4 meilleurs tokens lors de l'échantillonnage
            temperature=0.7  # Contrôle la diversité des sorties générées
        )

        # Décodage des identifiants générés en texte lisible
        decoded = self.tokenizer.batch_decode(generated_ids)

        # Extraction du texte généré tout en nettoyant les balises non désirées
        output = decoded[0].split("[/INST]")[1].replace("</s>", "").strip()

        # Si une liste de mots d'arrêt est fournie, tronquer la sortie à la première occurrence de ces mots
        if stop is not None:
            for word in stop:
                output = output.split(word)[0].strip()

        # Vérification que la sortie se termine correctement par une balise Markdown
        while not output.endswith("```"):
            output += "`"  # Ajout d'une apostrophe pour fermer correctement les snippets Markdown

        return output  # Retourne la sortie finale générée par le modèle

    # Propriété qui retourne un dictionnaire des paramètres identifiants du modèle
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model}  # Retourne un dictionnaire contenant le modèle utilisé

In [13]:
llm = CustomLLMMistral(model=model, tokenizer=tokenizer)# Création d'une instance de CustomLLMMistral avec le modèle et le tokenizer spécifiés


In [14]:
llm.invoke(input="quel est le role de l'ANSD du senegal?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'In Senegal, the National Agency for the Promotion of Employment and Professional Training (ANSEP, in its French acronym) was previously known as the National Agency for the Development of Small and Medium-Sized Enterprises (ANSD, in its French acronym). Both acronyms have different meanings, and I assume you meant to ask about ANSEP.\n\nThe ANSEP is a Senegalese government agency under the Ministry of Economy, Finance, and Planning. Its primary role is to promote employment and professional training in Senegal. It does this through various initiatives, such as:\n\n1. Vocational and technical training: ANSEP manages vocational and technical training centers throughout the country. These centers offer various courses to help young people and adults acquire the skills needed to find employment or start their own businesses.\n\n2. Business creation and support: ANSEP provides assistance to small and medium-sized enterprises (SMEs) in various ways, including business planning, access to cr

In [15]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


In [16]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [17]:
# Pour la recherche vectorielle
# Pour charger des fichiers PDF
# Pour diviser le texte en morceaux (non utilisé ici mais utile pour de grands documents)
# Pour créer une chaîne de questions-réponses

In [18]:
!pip install pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.3 MB/s eta 0:00:00


In [21]:
# prompt: link to my drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from langchain.document_loaders import PyPDFLoader
import pandas as pd
import os

# Liste des chemins vers vos fichiers PDF
pdf_files = ["/content/Chapitre 1- ETAT-STRUCTURE-POPULATION-Rapport-Provisoire-RGPH5_juillet2024_0.pdf","/content/Chapitre 2 - EDUCATION-Rapport-Provisoire-RGPH5_juillet2024_0.pdf","/content/Chapitre 3- ECONOMIE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf","/content/Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf","/content/Chapitre 5 - MORTALITE-Rapport-Provisoire-RGPH5_juillet2024.pdf"

    # Ajoutez d'autres fichiers PDF ici
]

# Créer une liste pour stocker le texte extrait
data_list = []

# Itérer sur chaque fichier PDF et charger le contenu
for pdf_file in pdf_files:
    if os.path.exists(pdf_file):  # Vérifier si le fichier existe
        loader = PyPDFLoader(pdf_file)
        documents = loader.load()

        # Extraire le texte de chaque document chargé
        for document in documents:
            data_list.append(document.page_content)
    else:
        print(f"Le fichier {pdf_file} n'existe pas.")

# Créer un DataFrame Pandas à partir du texte extrait
data = pd.DataFrame({"text": data_list})

# Accéder aux données avec les méthodes de pandas, par exemple, to_list()
data_list = data["text"].to_list()

# Afficher les résultats (facultatif)
print(data)

                                                  text
0                                   \n \n  \n \n1 1 \n
1     \n  \n \nII \n \n \n \n                      ...
2     \n  \n \nIII \n \nDirecteur général, Directeu...
3     \n  \n \nIV \n \nSOMMAIRE \nCHAPITRE I : ETAT...
4     \n  \n \nV \nListe des tableaux \nTableau I-1...
..                                                 ...
230   \n \n \n 41 \n41 \nDans l’ensemble, le niveau...
231   \n \n \n 42 \n42 \nGroupes \nd’âge \nStatut d...
232   \n \n \n 43 \n43 \nTableau V-26 : Répartition...
233   \n \n \n 44 \n44 \nCONCLUSION \nDe l’analyse ...
234                                       \n \n \n \n 

[235 rows x 1 columns]


In [41]:
# prompt: load this pdf file

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Chapitre 5 - MORTALITE-Rapport-Provisoire-RGPH5_juillet2024.pdf")
#pages = loader.load()

In [42]:
#from langchain.embeddings import HuggingFaceEmbeddings


# Load the PDF
#loader = PyPDFLoader("/content/Chapitre 12 - SITUATION-FEMMES-Rapport-Provisoire-RGPH5_juillet2024.pdf")
documents = loader.load()

In [43]:
# Split the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [44]:
#docs[7].page_content

In [45]:
!pip install sentence-transformers -q

In [46]:
!pip install faiss-gpu -q

In [53]:
# Create embeddings for retrieval
embeddings = HuggingFaceEmbeddings()

# Create a FAISS vectorstore for the documents
vectorstore = FAISS.from_documents(docs, embeddings)

NameError: name 'HuggingFaceEmbeddings' is not defined

In [54]:
!pip install langchain -q # install the package with the necessary imports
from langchain.embeddings import HuggingFaceEmbeddings # Import HuggingFaceEmbeddings

# Create embeddings for retrieval
embeddings = HuggingFaceEmbeddings()

# Create a FAISS vectorstore for the documents
vectorstore = FAISS.from_documents(docs, embeddings)

<ipython-input-54-9d8bd0ef7934>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-54-9d8bd0ef7934>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
bnn

In [ ]:
b

In [ ]:
 n

In [ ]:
 nn

In [56]:
vectorstore.search(query="quelle est l'esperance de vie des femmes?", k=1, search_type="similarity")# Recherche les 3 documents les plus similaires à la requête dans le vector store

[Document(metadata={'source': '/content/Chapitre 5 - MORTALITE-Rapport-Provisoire-RGPH5_juillet2024.pdf', 'page': 7}, page_content="Femmes de 15-49 ans 20,7 32,6 25,6 \nProportion de décès déclarés à l'état civil (%) \nEnsemble 47,4 43,7 45,8 \nHomme 68,3 26,3 47,4 \nFemme 66,7 21,3 43,7 \nTaux de mortalité par suicide 3,4 9,3 5,9 \nNombre de personnes décédées, disparues ou directement \ntouchées lors de catastrophes, pour 100 000 personnes 321 290 610")]

In [58]:

# Set up RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

# Ask a question
query = "repond en francais,  qu'esce que l'esperance de vie?"
response = qa_chain.run(query)

print(response)


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [59]:
from langchain.agents import create_json_chat_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
tools = []  # ou de la manière dont vous souhaitez la remplir
llm = []  # ou de la manière dont vous souhaitez la remplir
prompt = []  # ou de la manière dont vous souhaitez la remplir

In [60]:
!pip install openai-whisper transformers gtts

In [61]:
import whisper
from transformers import pipeline
from gtts import gTTS
import os

# Charger le modèle Whisper
model = whisper.load_model("base")

# Fonction principale pour traiter un message vocal
def traiter_message_vocal(fichier_audio):
    # Transcription de l'audio en texte
    result = model.transcribe(fichier_audio)
    texte_transcrit = result["text"]

    # Charger le modèle de Hugging Face pour la génération de réponses
    chatbot = pipeline("conversational", model="microsoft/DialoGPT-medium")

    # Générer une réponse avec Hugging Face
    reponse = chatbot(texte_transcrit)
    texte_reponse = reponse[0]['generated_text']

    # Convertir la réponse en audio et la sauvegarder
    tts = gTTS(text=texte_reponse, lang='fr')
    tts.save("reponse.mp3")

    # Jouer la réponse audio (ajustez selon votre système)
    os.system("start reponse.mp3") # Pour Windows

# Exemple d'utilisation
traiter_message_vocal("chemin/vers/votre_fichier_audio.mp3")

RuntimeError: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --ena

In [62]:
agent = create_json_chat_agent(
    tools = tools,
    llm = llm,
    prompt = prompt,
    stop_sequence = ["STOP"],
    template_tool_response = "{observation}"
)

AttributeError: 'list' object has no attribute 'input_variables'

In [63]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

<ipython-input-63-4cbb8db76ddd>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [64]:
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

NameError: name 'agent' is not defined

### Tests

In [ ]:
agent_executor.invoke({"input": "How much is 23 plus 17?"})

In [ ]:
agent_executor.invoke({"input": "What is the capital of France?"})

In [ ]:
agent_executor.invoke({"input": "Who was the inventor of the Radio?"})

In [ ]:
agent_executor.invoke({"input": "What is the double of the population of Madrid?"})

The following example usually fails due to limits in the model's reasoning capacity

In [ ]:
agent_executor.invoke({"input": "Who is older, Tom Hanks or Kevin Costner?"})